# Introduction to Music2leadsheet
* https://colab.research.google.com/drive/1iybFMeC09jZVxgP9evXW3IF_tS5p1nv6?usp=sharing

# Setup Library

In [1]:
import os
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
from librosa.effects import harmonic
from collections import Counter

# Separate vocals
* แยกเสียงร้องออกจากเพลง

In [2]:
filename = input().replace("\\" , "/")
filename
os.system(f"demucs -d cpu {filename}")

# Notes
* อ่านไฟล์เสียง
* เอาเสียงร้องที่มาแยกเป็นแปลเป็น Note 
* ใช้ความยาวของเสียงจุดเริ่มต้นของเสียงร้องถึงจุดสิ้นสุดเสียงเป็น 1 โน๊ต
* ต้องการเอาเสียงของโน๊ต(pitch) โดยเอาช่วงเวลามาทำของแต่ละโน๊ตมาทำ Fourier analysis โดยใช้ Fast Fourier Transform (time domain to frequency domain)โดยเลือก frequency ที่มากที่สุดเพื่อไปพยากรณ์ว่าเป็นโน๊ตตัวใด

In [ ]:
import librosa
import numpy as np

# อ่านไฟล์เสียงร้อง
audio_file = "D:\\Music2leadsheet\\separated\\htdemucs\\gonna\\vocals.wav"
y, sr = librosa.load(audio_file)

# ความถี่ของโน๊ตที่ถูกกำหนด
note_frequencies = {
    "A3": 220.0,
    "A#3": 233.1,
    "B3": 246.9,
    "C4": 261.6,
    "C#4": 277.2,
    "D4": 293.7,
    "D#4": 311.1,
    "E4": 329.6,
    "F4": 349.2,
    "F#4": 370.0,
    "G4": 392.0,
    "G#4": 415.3,
    "A4": 440.0 
}

# แยกเสียงร้องเป็นโน๊ตโดยใช้ onset detection
onset_frames = librosa.onset.onset_detect(y=y, sr=sr)
onset_times = librosa.frames_to_time(onset_frames, sr=sr)

# สร้างรายการเพื่อเก็บข้อมูลของโน๊ต
notes_data = []

# วนรอบสำหรับทุกโน๊ต
for i in range(len(onset_times) - 1):
    start_time = onset_times[i]
    end_time = onset_times[i + 1]
    note_audio = y[int(start_time * sr):int(end_time * sr)]

    # คำนวณ Fast Fourier Transform (FFT) ของแต่ละโน๊ต
    fft_result = np.fft.fft(note_audio)
    frequencies = np.fft.fftfreq(len(fft_result), 1.0 / sr)
    magnitude = np.abs(fft_result)
    max_frequency_index = np.argmax(magnitude)
    max_frequency = frequencies[max_frequency_index]

    # ค้นหาโน๊ตที่ใกล้เคียงที่สุด
    closest_note = None
    min_diff = float("inf")
    for note, frequency in note_frequencies.items():
        frequency_diff = abs(max_frequency - frequency)
        if frequency_diff < min_diff:
            min_diff = frequency_diff
            closest_note = note

    # เก็บข้อมูลของโน๊ต
    notes_data.append({
        "start_time": round(start_time , 1),
        "end_time": round(end_time , 1),
        "max_frequency": max_frequency,
        "note": closest_note
    })

# แสดงข้อมูลของโน๊ต
for note in notes_data:
    print("============================================")
    print("Start Time:", note["start_time"])
    print("End Time:", note["end_time"])
    print("Max Frequency:", note["max_frequency"])
    print("Note:", note["note"])


Start Time: 3.4
End Time: 4.1
Max Frequency: 0.0
Note: A3
Start Time: 4.1
End Time: 6.0
Max Frequency: 0.0
Note: A3
Start Time: 6.0
End Time: 7.4
Max Frequency: 350.68359375
Note: F4
Start Time: 7.4
End Time: 8.2
Max Frequency: 0.0
Note: A3
Start Time: 8.2
End Time: 8.4
Max Frequency: -592.0185501586526
Note: A3
Start Time: 8.4
End Time: 8.9
Max Frequency: 559.8632812499999
Note: A4
Start Time: 8.9
End Time: 9.1
Max Frequency: 263.18359375
Note: C4
Start Time: 9.1
End Time: 9.7
Max Frequency: 559.86328125
Note: A4
Start Time: 9.7
End Time: 10.2
Max Frequency: 389.6484375
Note: G4
Start Time: 10.2
End Time: 10.4
Max Frequency: 403.2581676136364
Note: G4
Start Time: 10.4
End Time: 11.5
Max Frequency: 417.26562499999994
Note: G#4
Start Time: 11.5
End Time: 12.5
Max Frequency: 350.14860733695645
Note: F4
Start Time: 12.5
End Time: 13.5
Max Frequency: -352.06787109375
Note: A3
Start Time: 13.5
End Time: 13.6
Max Frequency: 366.064453125
Note: F#4
Start Time: 13.6
End Time: 14.1
Max Frequenc

In [ ]:
# count note #
cnt = 1
for note in notes_data:
    print( note["note"] , " " , note["start_time"] , " " , note["end_time"] , " " , cnt  )
    cnt += 1
print ("count note : " , cnt )

A3   3.4   4.1   1
A3   4.1   6.0   2
F4   6.0   7.4   3
A3   7.4   8.2   4
A3   8.2   8.4   5
A4   8.4   8.9   6
C4   8.9   9.1   7
A4   9.1   9.7   8
G4   9.7   10.2   9
G4   10.2   10.4   10
G#4   10.4   11.5   11
F4   11.5   12.5   12
A3   12.5   13.5   13
F#4   13.5   13.6   14
A3   13.6   14.1   15
E4   14.1   14.4   16
F#4   14.4   14.5   17
A3   14.5   14.7   18
D4   14.7   15.5   19
G#4   15.5   16.9   20
A3   16.9   17.0   21
A3   17.0   18.8   22
A3   18.8   18.9   23
A#3   18.9   19.2   24
A4   19.2   19.3   25
C#4   19.3   19.4   26
C#4   19.4   19.6   27
C#4   19.6   19.8   28
D4   19.8   19.9   29
D#4   19.9   20.2   30
A4   20.2   20.6   31
A3   20.6   21.4   32
A3   21.4   22.0   33
A3   22.0   22.4   34
A4   22.4   22.7   35
A3   22.7   22.8   36
A3   22.8   22.8   37
A#3   22.8   23.4   38
C4   23.4   23.6   39
C#4   23.6   23.8   40
A#3   23.8   24.4   41
A3   24.4   24.5   42
A3   24.5   24.7   43
A4   24.7   25.2   44
G#4   25.2   25.4   45
A4   25.4   26.2   46
A

In [ ]:
notess = [
    "A#3",
    "C#4",
    "D#4",
    "F#4",
    "G#4",
]
tmp = []
for note in notes_data:
    if note["note"] in notess :
        tmp.append(note["note"])       
cnt = len(set(tmp))  
print ("count note : " , cnt)

count note :  5


# Keys
* เอาโน๊ตทั้งหมดที่พยากรออกมาดูว่าใช้โน๊ตตัวไหนมากที่สุดเอา top 5 มาเทียบกับทฤษฏีดนตรีว่าควรใช้ตัวไหน

In [ ]:
keys = {
    0 : "C" ,
    1 : "G" ,
    2 : "D" ,
    3 : "A" ,
    4 : "E" ,
    5 : "B" ,
    6 : "F#" ,
    7 : "C#" ,
}
notess = [
    "A#3",
    "C#4",
    "D#4",
    "F#4",
    "G#4",
]
tmp = []
for note in notes_data:
    if note["note"] in notess :
        tmp.append(note["note"])       
cnt = len(set(tmp))
print(keys[cnt])

B


# Cords
* แบ่งโน๊ตออกเป็น 4 ห้องเอาโน๊ตทั้งหมดที่พยากรออกมาดูว่าใช้โน๊ตตัวไหนมากที่สุดเอา top 3 มาเทียบกับทฤษฏีดนตรีว่าควรใช้ตัวไหน

# Generate Datasheet & Sound Midi

In [ ]:
from music21 import stream, note, duration

def create_midi_from_notes_data(data):

    s = stream.Stream()
    note_preprocess_data = []
    for note_info in data:
        print(note_info["note"] , " : " , abs(note_info["start_time"] - note_info["end_time"]))
        note_preprocess_data.append(note.Note(note_info["note"] , quarterLength = abs(note_info["start_time"] - note_info["end_time"])))
    
    s.append(note_preprocess_data)
    s.write('midi' , fp='my_melody.midi')

# TEST MELODY

In [39]:
print(notes_data)
create_midi_from_notes_data(notes_data)

[{'start_time': 3.4, 'end_time': 4.1, 'max_frequency': 0.0, 'note': 'A3'}, {'start_time': 4.1, 'end_time': 6.0, 'max_frequency': 0.0, 'note': 'A3'}, {'start_time': 6.0, 'end_time': 7.4, 'max_frequency': 350.68359375, 'note': 'F4'}, {'start_time': 7.4, 'end_time': 8.2, 'max_frequency': 0.0, 'note': 'A3'}, {'start_time': 8.2, 'end_time': 8.4, 'max_frequency': -592.0185501586526, 'note': 'A3'}, {'start_time': 8.4, 'end_time': 8.9, 'max_frequency': 559.8632812499999, 'note': 'A4'}, {'start_time': 8.9, 'end_time': 9.1, 'max_frequency': 263.18359375, 'note': 'C4'}, {'start_time': 9.1, 'end_time': 9.7, 'max_frequency': 559.86328125, 'note': 'A4'}, {'start_time': 9.7, 'end_time': 10.2, 'max_frequency': 389.6484375, 'note': 'G4'}, {'start_time': 10.2, 'end_time': 10.4, 'max_frequency': 403.2581676136364, 'note': 'G4'}, {'start_time': 10.4, 'end_time': 11.5, 'max_frequency': 417.26562499999994, 'note': 'G#4'}, {'start_time': 11.5, 'end_time': 12.5, 'max_frequency': 350.14860733695645, 'note': 'F